# Trabajo 02: Modelos de riesgo de crédito con RNA

## Pre-procesamiento de los datos

En este segmento explicamos todo el análisis exploratorio de datos, perfilamiento y preparación de los datos para su uso en una red neuronal

Importamos los datos

In [24]:
# data set from
# https://www.kaggle.com/api/v1/datasets/download/ranadeep/credit-risk-dataset?dataset_version_number=3

import pandas as pd

df_raw = pd.read_csv("loan.csv") # Cargar datos como df_raw

C:\Users\Kopynft\AppData\Local\Temp\ipykernel_12332\4139838488.py:6: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("loan.csv") # Cargar datos como df_raw


Leemos las columnas de los datos

In [25]:
print(df_raw.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

Hacemos un análisis de los datos presentes y decidimos qué datos se van a quitar y cuáles a utilizar para el entrenamiento de la red neuronal. La siguiente información es extraída del mismo sitio de donde se obtuvieron los [datos](https://www.kaggle.com/datasets/ranadeep/credit-risk-dataset/data). En primera instancia se ignoran las columnas que tienen muchos datos nulos, se considera "muchos" a aquellas columnas que tengan más de 50 nulos

In [26]:
null_counts = df_raw.isnull().sum() # Sumar nulos por columna

print("Nulos en cada columna:")
print(null_counts)

Nulos en cada columna:
id                       0
member_id                0
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                     ...  
all_util            866007
total_rev_hi_lim     70276
inq_fi              866007
total_cu_tl         866007
inq_last_12m        866007
Length: 74, dtype: int64


Se eliminan entonces las siguientes columnas:


"emp_title", "emp_length", "desc", "title", "mths_since_last_delinq", "mths_since_last_record", "revol_util", "last_pymnt_d", "next_pymnt_d", "last_credit_pull_d", "collections_12_mths_ex_med", "mths_since_last_major_derog", "annual_inc_joint", "dti_joint", "verification_status_joint", "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_il_6m", "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m"

In [27]:
df_nonull = df_raw.drop(["emp_title", "emp_length", "desc", "title", "mths_since_last_delinq", "mths_since_last_record", "revol_util", "last_pymnt_d", "next_pymnt_d", "last_credit_pull_d", "collections_12_mths_ex_med", "mths_since_last_major_derog", "annual_inc_joint", "dti_joint", "verification_status_joint", "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_il_6m", "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m"], axis=1)

Nos quedan ahora las siguientes columnas:

In [28]:
print(df_nonull.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'purpose', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code',
       'application_type', 'acc_now_delinq'],
      dtype='object')


De esas columnas se decide eliminar entonces información considerada no significativa, ya sea porque es información personal, implica añadir complejidad financiera a la red, se considera redundante o aplica solo a deudas que se consideran "perdidas". Se eliminan:

addr_state: The state provided by the borrower in the loan application.\
collection_recovery_fee: Post charge off collection fee.\
funded_amnt: The total amount committed to that loan at that point in time.\
funded_amnt_inv: The total amount committed by investors for that loan at that point in time.\
grade: LC assigned loan grade.\
id: A unique LC assigned ID for the loan listing.\
initial_list_status: The initial listing status of the loan. Possible values are W, F.\
issue_d: The month which the loan was funded.\
member_id: A unique LC assigned Id for the borrower member.\
out_prncp: Remaining outstanding principal for total amount funded.\
out_prncp_inv: Remaining outstanding principal for portion of total amount funded by investors.\
pymnt_plan: Indicates if a payment plan has been put in place for the loan.\
recoveries: post charge off gross recovery\
total_pymnt: Payments received to date for total amount funded.\
total_pymnt_inv: Payments received to date for portion of total amount funded by investors.\
total_rec_int: Interest received to date.\
total_rec_late_fee: Late fees received to date.\
total_rec_prncp: Principal received to date.\
url: URL for the LC page with listing data.\
verification_status: NO APARECE EN EL DICCIONARIO.\
zip_code: The first 3 numbers of the zip code provided by the borrower in the loan application.

"addr_state", "collection_recovery_fee", "funded_amnt", "funded_amnt_inv", "grade", "id", "initial_list_status", "issue_d", "member_id", "out_prncp", "out_prncp_inv", "pymnt_plan", "recoveries", "total_pymnt", "total_pymnt_inv", "total_rec_int", "total_rec_late_fee", "total_rec_prncp", "url", "verification_status", "zip_code"

In [29]:
df_clean = df_nonull.drop(["addr_state", "collection_recovery_fee", "funded_amnt", "funded_amnt_inv", "grade", "id", "initial_list_status", "issue_d", "member_id", "out_prncp", "out_prncp_inv", "pymnt_plan", "recoveries", "total_pymnt", "total_pymnt_inv", "total_rec_int", "total_rec_late_fee", "total_rec_prncp", "url", "verification_status", "zip_code"], axis=1)

Quedan entonces las siguientes columans:

acc_now_delinq: The number of accounts on which the borrower is now delinquent.\
application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers.\
annual_inc: The self-reported annual income provided by the borrower during registration.\
delinq_2yrs: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past [...]\
dti: A ratio calculated using the borrower's total monthly debt payments on the total debt obligations [...]\
earliest_cr_line: The date the borrower's earliest reported credit line was opened.\
home_ownership: The home ownership status provided by the borrower during registration.\
inq_last_6mths: The number of inquiries in past 6 months (excluding auto and mortgage inquiries).\
installment: The monthly payment owed by the borrower if the loan originates.\
int_rate: Interest Rate on the loan.\
last_pymnt_amnt: Last total payment amount received.\
loan_amnt: The listed amount of the loan applied for by the borrower.\
loan_status: Current status of the loan.\
open_acc: The number of open credit lines in the borrower's credit file.\
policy_code: publicly available policy_code=1 new products not publicly available policy_code=2.\
pub_rec: Number of derogatory public records.\
purpose: A category provided by the borrower for the loan request.\
revol_bal: Total credit revolving balance.\
sub_grade: LC assigned loan subgrade.\
term: The number of payments on the loan. Values are in months and can be either 36 or 60.\
total_acc: The total number of credit lines currently in the borrower's credit file.\

Veamos ahora los tipos de datos, ya que algunos como "earliest_cr_line" no son de tipo numérico sino de tipo fecha, así que deben utilizarse una conversión para facilitar el entrenamiento del modelo

Para las columnas: term, sub_grade, home_ownership, loan_status, purpose, earliest_cr_line y application_type se debe hacer un procesamiento externo para convertirlos a datos más sencillos.

Para "term" que son las cuotas, vienen en "36 months" y "60 months" así que se retirará la parte de tipo texto y se dejará solo el valor numérico.

"sub_grade" cuenta con las categorías de A a G y sub categorías de 1 a 5, así que se cambiarán consecuentemente desde A1 hasta G5 por valores numéricos del 1 al 35 (A1: 1, A2: 2, ..., B3: 8, B4: 9, ..., G5: 35).

"home_ownsership" especifica si quien solicita la deuda posee casa, tiene las categorías: Mortgage (Hipoteca), Rent (Arrienda), Own (Propia), Other (Otra), None (Ninguna). Nos interesan los criterios de si posee o no y si tiene arrendamiento, así que se hará la siguiente conversión.

Mortgage: 0 porque la persona tiene una deuda que pagar.\
Own: 2 porque la persona posee vivienda propia y no debe gastar en arriendo.\
Los demás se les asigna un valor de 1, para posicionarlos en medio.

Respecto a "purpose" se les asignará valores de 0 a 13 del criterio más repetido al menos repetido (debt_consolidation: 0, credit_card: 1, home_improvement: 2, other: 3, major_purchase: 4
small_business: 5, car: 6, medical: 7, moving: 8, vacation: 9, house: 10, wedding: 11, renewable_energy: 12, educational: 13)

"earliest_cr_line" es un dato que originalmente está en tipo fecha así que decide separarse el año y el mes para guardarlos como tipo numérico

Por otro lado "application_type" tiene el criterio "INDIVIDUAL" y "JOINT" por lo que convertimos INDIVIDUAL a 0 y JOINT a 1

Finalmente "loan_status" es la variable que nos interesa ya que estamos midiendo la capacidad de un cliente de pagar el crédito, así que le cambiaremos el nombre a "target" asignaremos los siguientes valores:

Fully Paid y Current: 1 puesto que signifca que han cumplido o están cumpliendo con los pagos.\
Los demás valores se les asignará 0 ya que si bien no la han pagado ni están al día, tampoco se considera perdida así que todavía puede que paguen la deuda.


In [31]:

# 1. Primero extraemos la parte numérica del "term"
df_clean["term"] = df_clean["term"].str.extract('(\d+)').astype(int)

# 2. Ajustamos "subgrade"
# Creamos un diccionario
mapeo_subgrade = { 'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5,
                   'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10,
                   'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15,
                   'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20,
                   'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25,
                   'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30,
                   'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35,
}

# Mapeamos los "subgrades" a sus valores numéricos
df_clean["sub_grade"] = df_clean["sub_grade"].map(mapeo_subgrade)

# 3. Ajustamos "home_ownership"
#  Creamos un diccionario
mapeo_house = {
    'MORTGAGE': 0,
    'OWN': 2,
    'other': 1  # Asignamos un valor por defecto a las demás categorías
}


# Mapeamos las categorías a sus valores numéricos
df_clean["home_ownership"] = df_clean["home_ownership"].map(mapeo_house).fillna(1)

# 4. Ajustamos "purpose"
# Creamos un diccionario
mapeo_purpose = { "debt_consolidation": 0, "credit_card": 1,
                  "home_improvement": 2, "other": 3, "major_purchase": 4,
                  "small_business": 5, "car": 6, "medical": 7, "moving": 8,
                  "vacation": 9, "house": 10, "wedding": 11,
                  "renewable_energy": 12, "educational": 13
}
# Mapeamos los "pupose" a sus valores numéricos
df_clean["purpose"] = df_clean["purpose"].map(mapeo_purpose)

# 5. Ajustamos "earliest_cr_line"
# Convertimos de tipo Object a tipo Fecha con Mes-Año
df_clean["earliest_cr_line"] = pd.to_datetime(df_clean["earliest_cr_line"], format='%b-%Y')

# Extraemos el mes y el año
df_clean["month"] = df_clean["earliest_cr_line"].dt.month
df_clean["year"] = df_clean["earliest_cr_line"].dt.year

# Finalmente ignoramos la columna original
df_clean = df_clean.drop("earliest_cr_line", axis=1)

# 6. Ajustamos "application_type"
# Creamos un diccionario
mapeo_aptype = { "INDIVIDUAL": 0,  "JOINT": 1}

# Mapeamos los "application_type" a sus valores numéricos
df_clean["application_type"] = df_clean["application_type"].map(mapeo_aptype)

# 7. Ajustamos y renombramos "loan_status"
# Creamos un diccionario
mapeo_lnst = {"Current": 1,
              "Fully Paid": 1,
              "other": 0}

# Mapeamos los "loan_status" a sus valores numéricos
df_clean["loan_status"] = df_clean["loan_status"].map(mapeo_lnst).fillna(0)

# Cambiamos el tipo a booleano
df_clean["loan_status"] = df_clean["loan_status"].astype(bool) 

# Y finalmente renombramos
df_clean = df_clean.rename(columns={"loan_status": 'target'})

Finalmente quitamos los NaN que hayan quedado en las filas

In [32]:
df_clean = df_clean.dropna()

Con esto hemos concluido todo los pre-procesamientos necesarios a la información original y ahora podemos prepararla para entrenar el modelo.

## Procesamiento de los datos y entreamiento del modelo.

Dado que existe un gran desbalance entre la información resultante, puesto que existe criterios muy diferentes entre quienes logran pagar y quienes no, así mismo con la información que se tiene de ellos (por ejemplo hay muy pocos quienes tienen casa o quienes han logrado existosamente pagar su créditos) así que haremos una técnica de balanceo llamada Tomek Link permitiendo una mejor identificación, adicionalmente se usa SMOTE para darle una mejor representación a las categorías minoritarias

In [33]:
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

# Separar características y target
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# Submuestreo utilizando TomekLinks para eliminar muestras cercanas
tl = TomekLinks()
X_under, y_under = tl.fit_resample(X, y)

# Sobremuestreo de la clase minoritaria hasta el 30% de la clase mayoritaria
smote = SMOTE(sampling_strategy=0.3, random_state=40)
X_balanced, y_balanced = smote.fit_resample(X_under, y_under)

# Combinar las características balanceadas con el target
df_training = pd.concat([pd.DataFrame(X_balanced, columns=X.columns), pd.Series(y_balanced, name='target')], axis=1)

# Guardamos los datasets para posterior uso
df_clean.to_csv("datos_limpios.csv", index=False)
df_training.to_csv("datos_balanceados.csv", index=False)

Después del ajuste y el balanceo de datos se procede ahora a el entreamiento del modelo. La red neuronal se compone de dos capas de 32 y 16 neuronas con activación relu con la capa de salida de una neurona con una activación sigmoide, la compilación del modelo está compuesta por el optimizador Adaptive Moment Estimation, la función de perdida es binary_crossentropy, útil en este caso por ser una clasificación binaria, y las métricas de optimización se decidió por el ROC siglas de área bajo la curva, PR Calcula el área bajo la curva de precisión-recall, útil en problemas con clases desbalanceadas.


In [2]:
df_training = pd.read_csv("datos_balanceados.csv")

In [ ]:
from sklearn.utils import class_weight
import numpy as np
# Definir y compilar el modelo
from tensorflow import keras
from tensorflow.keras import layers

# Supongamos que dEntrenamiento_balanced contiene todas las columnas
X = df_training.drop("target", axis=1)
y = df_training["target"]

# Normalización opcional
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Calcular pesos de clase para manejar la clase minoritaria
classes = np.unique(y_train)
weights = class_weight.compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))


model = keras.Sequential([
    layers.Dense(32, activation="relu", input_shape=(X_scaled.shape[1],)),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC(name="auc", curve="ROC"),keras.metrics.AUC(name="pr_auc", curve="PR")])
y_train = y_train.reset_index(drop=True)
X_train = pd.DataFrame(X_train).reset_index(drop=True)

model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, class_weight=class_weights)

c:\Users\Kopynft\Documents\UNAL\RNAABI\Trabajo 02\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - auc: 0.8291 - loss: 0.5015 - pr_auc: 0.9394
Epoch 2/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - auc: 0.8902 - loss: 0.4129 - pr_auc: 0.9609
Epoch 3/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - auc: 0.9045 - loss: 0.3849 - pr_auc: 0.9661
Epoch 4/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - auc: 0.9093 - loss: 0.3735 - pr_auc: 0.9679
Epoch 5/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - auc: 0.9117 - loss: 0.3681 - pr_auc: 0.9687
Epoch 6/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - auc: 0.9141 - loss: 0.3628 - pr_auc: 0.9695
Epoch 7/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - auc: 0.9167 - loss: 0.3573 - pr_auc: 0.9704
Epoch 8/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - auc: 0.9187 - loss: 0.3524 - pr_auc: 0.9712
Epoch 9/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step - auc: 0.9203 - loss: 0.3489 - pr_auc: 0.9718
Epoch 10/30
25197/25197 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - auc: 0.9211 -

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_probs = model.predict(X_test)
y_pred_classes = (y_pred_probs >= 0.5).astype(int)  # Ajustar umbral
acc = accuracy_score(y_test, y_pred_classes)
prec = precision_score(y_test, y_pred_classes)
rec = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)
print(acc, prec, rec, f1)

6300/6300 ━━━━━━━━━━━━━━━━━━━━ 5s 768us/step
0.8541195380402429 0.9371652360216475 0.8686443409973049 0.9016047862515727


In [37]:
fila_ejemplo = X_test[0:1]  # Toma una fila como ejemplo
prob_pred = model.predict(fila_ejemplo)
clase_pred = (prob_pred >= 0.5).astype(int)

print("Probabilidad predicción:", prob_pred)
print("Clase predicha:", clase_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Probabilidad predicción: [[0.9448362]]
Clase predicha: [[1]]


In [38]:
# Datos de entrada
X = df_training.drop("target", axis=1)
y = df_training["target"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_pred_probs = model.predict(X_scaled)
y_pred_classes = (y_pred_probs >= 0.5).astype(int)  # Ajustar umbral
acc = accuracy_score(y, y_pred_classes)
prec = precision_score(y, y_pred_classes)
rec = recall_score(y, y_pred_classes)
f1 = f1_score(y, y_pred_classes)
print(acc, prec, rec, f1)

31497/31497 ━━━━━━━━━━━━━━━━━━━━ 23s 734us/step
0.8544972705029775 0.9372784538933685 0.8689988662321632 0.9018481322907457


In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# Supongamos que dEntrenamiento_balanced contiene todas las columnas
X = df_training.drop("target", axis=1)
y = df_training["target"]

# Normalización opcional
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Calcular pesos de clase para manejar la clase minoritaria
classes = np.unique(y_train)
weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, weights))

# Convertir X_train a DataFrame
X_train = pd.DataFrame(X_train).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Definir y compilar el modelo sin cambiar las métricas existentes
model = keras.Sequential([
    layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.AUC(name="auc", curve="ROC"),
        keras.metrics.AUC(name="pr_auc", curve="PR")
    ]
)

# Limitar los epochs cuando no mejore (monitor: val_loss)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True
)

# Entrenar el modelo con early stopping
history = model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=32,
    verbose=1,
    class_weight=class_weights,
    validation_split=0.2,
    callbacks=[early_stop]
)

# ---------------------------
# Evaluar la influencia de cada variable (Permutation Feature Importance)
# ---------------------------
X_test_df = pd.DataFrame(X_test, columns=X.columns).reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Función para evaluar rendimiento (ROC-AUC, PR-AUC)
def evaluate_performance(model, X_data, y_data):
    y_probs = model.predict(X_data).ravel()
    roc = roc_auc_score(y_data, y_probs)
    pr = average_precision_score(y_data, y_probs)
    return roc, pr

# Rendimiento base
baseline_roc, baseline_pr = evaluate_performance(model, X_test_df, y_test)

feature_importances = []

for col in X_test_df.columns:
    # Guardar la columna original
    original_col = X_test_df[col].copy()

    # Barajar (shuffle) la columna
    X_test_df[col] = np.random.permutation(X_test_df[col].values)

    # Evaluar el rendimiento tras el shuffle
    shuffled_roc, shuffled_pr = evaluate_performance(model, X_test_df, y_test)

    # Restaurar la columna original
    X_test_df[col] = original_col

    # Cambio en rendimiento
    drop_roc = baseline_roc - shuffled_roc
    drop_pr = baseline_pr - shuffled_pr

    feature_importances.append((col, drop_roc, drop_pr))

# Ordenar de mayor impacto (peor si se shufflea) a menor
feature_importances.sort(key=lambda x: (x[1], x[2]), reverse=True)

print("\nInfluencia de cada variable (Ordenadas de mayor a menor impacto):")
for feat, d_roc, d_pr in feature_importances:
    print(f"Variable: {feat} | ΔROC-AUC: {d_roc:.4f} | ΔPR-AUC: {d_pr:.4f}")

c:\Users\Kopynft\Documents\UNAL\RNAABI\Trabajo 02\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - auc: 0.8173 - loss: 0.5169 - pr_auc: 0.9352 - val_auc: 0.8708 - val_loss: 0.4666 - val_pr_auc: 0.9547
Epoch 2/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - auc: 0.8752 - loss: 0.4384 - pr_auc: 0.9565 - val_auc: 0.8872 - val_loss: 0.3962 - val_pr_auc: 0.9605
Epoch 3/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - auc: 0.8924 - loss: 0.4084 - pr_auc: 0.9622 - val_auc: 0.9046 - val_loss: 0.3703 - val_pr_auc: 0.9663
Epoch 4/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - auc: 0.9052 - loss: 0.3826 - pr_auc: 0.9669 - val_auc: 0.9100 - val_loss: 0.3438 - val_pr_auc: 0.9682
Epoch 5/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 51s 3ms/step - auc: 0.9091 - loss: 0.3743 - pr_auc: 0.9680 - val_auc: 0.9102 - val_loss: 0.3742 - val_pr_auc: 0.9681
Epoch 6/40
20158/20158 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - auc: 0.9115 - loss: 0.3691 - pr_auc: 0.9687 - val_auc: 0.9143 - val_loss: 0.3456 - val_pr_auc: 0.9695
Epoch 7/40
20158/20158 ━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch, Objective
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Datos de entrada
X = df_training.drop("target", axis=1)
y = df_training["target"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Ajustar pesos de clase
classes = np.unique(y_train)
weights = class_weight.compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

# Convertir a DataFrame y Series con índices reiniciados
X_train = pd.DataFrame(X_train).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Definición de función de construcción de modelo para Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    # Número de capas ocultas
    for i in range(hp.Int("num_layers", min_value=1, max_value=3)):
        model.add(layers.Dense(
            units=hp.Int("units_" + str(i), min_value=16, max_value=128, step=16),
            activation="relu",
            kernel_regularizer=keras.regularizers.l2(hp.Float("l2_" + str(i), 0.0001, 0.01, sampling="log"))
        ))
        model.add(layers.Dropout(hp.Float("dropout_" + str(i), 0.1, 0.5, step=0.1)))

    model.add(layers.Dense(1, activation="sigmoid"))

    # Tasa de aprendizaje variable
    lr = hp.Choice("learning_rate", values=[1e-3, 5e-4, 1e-4])
    opt = keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=[keras.metrics.AUC(name="auc", curve="ROC"),
        keras.metrics.AUC(name="pr_auc", curve="PR")])
    return model

# Configuración de la búsqueda aleatoria
tuner = RandomSearch(
    build_model,
    objective=Objective("val_auc", direction="max"),
    max_trials=5,
    executions_per_trial=1,
    directory="my_tuner_dir",
    project_name="hyperparam_opt"
)

stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

# Búsqueda de hiperparámetros
tuner.search(
    X_train, y_train,
    epochs=40,
    validation_split=0.2,
    callbacks=[stop_early],
    class_weight=class_weights,
    verbose=0
)

# Obtener el mejor modelo y guardarlo
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save("modelo_final.keras")

# Imprimir la arquitectura del mejor modelo
best_model.summary()